In [1]:
from typing import List, Tuple, Union

In [2]:
import numpy as np
from transformers import BertTokenizer, BertForMaskedLM

from string import punctuation
from nltk.corpus import stopwords

punctuation += '«»—…“”*№–'
punctuation = set(punctuation)
stopwords = set(stopwords.words('russian'))


class Guess:
    """
    Этот класс используется для структурированного хранения предсказаний модели для маскированных токенов.
    """

    def __init__(self,
                 words: List[str],
                 probs: Union[List[float], np.ndarray]) -> None:
        self.words = words
        self.probs = np.array(probs) if isinstance(probs, list) else probs

    @classmethod
    def from_model_predictions(cls,
                               probs: np.ndarray,
                               tokenizer: BertTokenizer,
                               max_len: int = 1000):
        probs_with_indices = list(enumerate(probs))
        best_probs_with_indices = sorted(probs_with_indices, key=lambda x: x[1], reverse=True)

        best_words = {}
        for i, prob in best_probs_with_indices:
            word = tokenizer.convert_ids_to_tokens(i).lower()
            if word in punctuation | stopwords or word in best_words:
                continue

            best_words[word] = prob

            if len(best_words) == max_len:
                break

        best_probs = list(best_words.values())
        best_words = list(best_words.keys())

        return cls(best_words, best_probs)

    def get_best_guess(self) -> Union[Tuple[str, float], None]:
        return self.words[0], self.probs[0] if bool(self) else None

    def get_n_best_guesses(self, n: int = 5) -> Union[List[str], None]:
        return self.words[:n] if bool(self) else None

    def __bool__(self) -> bool:
        return bool(self.words) and self.probs.shape[0] == len(self.words)

    def __str__(self) -> str:
        return '\n'.join([f'{word}: {prob:.2f}' for word, prob in zip(self.words, self.probs)])

    def __len__(self) -> int:
        return self.probs.shape[0]

    def __add__(self, x):
        if not self and not x:
            return Guess([], [])
        if not self:
            return x
        if not x:
            return self

        combined_words = set(self.words) & set(x.words)
        if not combined_words:
            return Guess([], [])

        new_words = {}

        i, j = 0, 0
        while i < len(self) or j < len(x):
            if j == len(x) or (i < len(self) and self.probs[i] > x.probs[j]):
                if self.words[i] not in combined_words:
                    i += 1
                    continue
                new_words[self.words[i]] = (new_words.get(self.words[i], self.probs[i]) + self.probs[i]) / 2
                i += 1
            else:
                if x.words[j] not in combined_words:
                    j += 1
                    continue
                new_words[x.words[j]] = (new_words.get(x.words[j], x.probs[j]) + x.probs[j]) / 2
                j += 1

        new_len = min(len(self), len(x))
        sorted_items = sorted(new_words.items(), key=lambda x: x[1], reverse=True)[:new_len]
        new_words = [x[0] for x in sorted_items]
        new_probs = np.array([x[1] for x in sorted_items])

        return Guess(new_words, new_probs)

In [3]:
g1 = Guess(['мама', 'мыла', 'раму'], np.array([0.5, 0.3, 0.2]))

In [4]:
print(g1)

мама: 0.50
мыла: 0.30
раму: 0.20


In [5]:
g1.get_best_guess()

('мама', 0.5)

In [6]:
g2 = Guess(['мама', 'мыла', 'раму'], [0.7, 0.15, 0.15])

In [7]:
print(g1 + g2)

мама: 0.60
мыла: 0.22
раму: 0.17


In [8]:
g3 = Guess([], [])

In [9]:
bool(g3)

False

In [10]:
str(g3)

''

In [11]:
len(g3)

0

In [12]:
import torch

from abc import ABC, abstractmethod


class MLMPredictor(ABC):
    """
    Это абстрактный класс для предиктора MLM-модели.
    """
    
    def __init__(self,
                 bert_path: str = 'rubert_cased_L-12_H-768_A-12_pt',
                 device: str = 'cpu') -> None:
        self.tokenizer = BertTokenizer.from_pretrained(bert_path, do_lower_case=False)
        self.mask_token = self.tokenizer.mask_token
        
        self.model = BertForMaskedLM.from_pretrained(bert_path)
        self.model.eval()
        
        self.device = torch.device(device)
        self.model.to(self.device)
    
    @abstractmethod
    def tokenize(*args, **kwargs):
        pass
    
    @abstractmethod
    def predict(*args, **kwargs):
        pass

In [13]:
from scipy.special import softmax

from overrides import overrides


class BERTMLMPredictor(MLMPredictor):
    """
    Этот класс позволяет получить предсказания MLM-модели для маскированного токена.
    """
    
    @overrides
    def tokenize(self, text: str) -> Tuple[List[str], int]:
        tokens = self.tokenizer.tokenize(text.strip(), add_special_tokens = True)
        
        mask_indices = [i for i, token in enumerate(tokens) if token == self.mask_token]
        if len(mask_indices) == 0:
            raise IOError(f'The mask token is not found in model input: {text}.')
        if len(mask_indices) > 1:
            raise IOError(f'The mask token occurs more than once in model input: {text}.')
        mask_idx = mask_indices[0]
    
        return tokens, mask_idx
        
    @overrides
    def predict(self,
                text: Union[str, List[str]]) -> Guess:
        tokens, mask_idx = self.tokenize(text) if isinstance(text, str) \
            else self.tokenize(' '.join(text))
        
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        
        with torch.no_grad():
            model_output = self.model(torch.tensor([token_ids]).to(self.device))[0][0]
        
        masked_probs = softmax(model_output[mask_idx].cpu().numpy())
        guess = Guess.from_model_predictions(masked_probs, self.tokenizer)
        
        return guess

In [14]:
class GameHandler:
    """
    Этот класс отвечает за взаимодействие с пользователем и координирует работу остальных классов.
    """
    
    def __init__(self,
                 bert_path: str = 'rubert_cased_L-12_H-768_A-12_pt',
                 max_rounds: int = 10,
                 prob_threshold: float = 0.5,
                 device: str = 'cpu') -> None:
        self.predictor = BERTMLMPredictor(bert_path, device)
        self.guess = Guess([], [])
        self.max_rounds = max_rounds
        self.prob_threshold = prob_threshold
    
    def __update_guess(self, text: str) -> None:
        new_guess = self.predictor.predict(text)
        self.guess += new_guess
    
    def __play_round(self) -> None:
        text = input()
        self.__update_guess(text)
    
    def play_game(self, n: int = 5) -> None:
        for _ in range(self.max_rounds):
            self.__play_round()
            
            if not self.guess:
                print(f'Вы меня запутали, я сдаюсь.')
                return
            
            best_word, best_prob = self.guess.get_best_guess()
            if best_prob > self.prob_threshold or len(self.guess) == 1:
                print(f'Я думаю, вы загадали слово "{best_word}".')
                return
        
        best_guesses = self.guess.get_n_best_guesses(n)
        if len(best_guesses) == 1:
            print(f'Я думаю, вы загадали слово "{best_guesses[0]}".')
            return
        
        best_guesses = list(map(lambda x: '"' + x + '"', best_guesses))
        
        print(f'Вот мои лучшие догадки: {", ".join(best_guesses)}.')

In [15]:
game_handler = GameHandler()

In [16]:
game_handler.play_game()

На Гавайских островах активен [MASK] Килауэа.
Геологические образования на поверхности Земли, где магма выходит на поверхность, называют словом [MASK].
Бурлящий [MASK] начал медленно извергать раскалённую лаву.
На северо-западе Мордора, посреди плато Горгорот, находится [MASK] Ородруин, где Саурон выковал Кольцо Всевластья.
Древний [MASK] Ородруин (в переводе с синдарина это означает "гора алого пламени") поглотил Голлума вместе с Кольцом Всевластья.
На восточном побережье Сицилии находится действующий [MASK] Этна.
[MASK], который ни разу не извергался в последние 10 тысяч лет, считается потерявшим свою активность, потухшим.
В тот страшный день произошло извержение. [MASK] выплеснул на поверхность тонны пылающей лавы.
Самый большой [MASK] в мире - это Мауна-Лоа.
На границе Чили и Аргентины находится самый высокий действующий [MASK] на Земле.
Я думаю, вы загадали слово "вулкан".
